# IMPORT

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

In [62]:
from rolling_func import apply_rolling_aggregation
from feature_engineering_utils import compute_bureau_features, compute_bureau_balance_features, compute_application_features, bb_features_per_account, agg_bb_to_customer

# LOAD DATA

In [4]:
data_dir = r'E:\Project\Dissertation'

In [5]:
df_train = pd.read_csv(rf'{data_dir}\home-credit-default-risk\application_train.csv')
df_bureau  = pd.read_csv(rf'{data_dir}\home-credit-default-risk\bureau.csv')
df_bureau_bal  = pd.read_csv(rf'{data_dir}\home-credit-default-risk\bureau_balance.csv')

In [6]:
df_bureau = df_bureau.rename(columns={"SK_ID_BUREAU":"SK_BUREAU_ID"})
df_bureau_bal = df_bureau_bal.rename(columns={"SK_ID_BUREAU":"SK_BUREAU_ID"})

# FEATURE ENGINEERING

## Bureau info

In [79]:
df_bureau.head()

,SK_ID_CURR,SK_BUREAU_ID,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


### Fill missing

In [80]:
list_bureau_fillna = [
    'AMT_CREDIT_MAX_OVERDUE'
    ,'CNT_CREDIT_PROLONG'
    ,'AMT_CREDIT_SUM'
    ,'AMT_CREDIT_SUM_DEBT'
    ,'AMT_CREDIT_SUM_LIMIT'
    ,'AMT_CREDIT_SUM_OVERDUE'
    ,'AMT_ANNUITY']

for ft in list_bureau_fillna:
    df_bureau[ft] = df_bureau[ft].fillna(0)

In [81]:
bureau_features = compute_bureau_features(df_bureau, windows=[30, 90, 180, 360, 720])

In [82]:
df_bureau[['SK_ID_CURR']].nunique()

SK_ID_CURR    305811
dtype: int64

In [83]:
bureau_features.shape

(305811, 196)

In [84]:
bureau_features[bureau_features['w30_n_active_loans']>2]

,SK_ID_CURR,w30_n_active_loans,w30_n_closed_loans,w30_n_total_loans,w30_active_ratio,w30_type_diversity_shannon,w30_currency_diversity,w30_total_debt,w30_total_exposure,w30_utilization_portfolio,...,w720_last_loan_debt,w720_last_loan_overdue_amt,w720_last_loan_type,w720_debt_concentration_active,w720_laddering_share,w720_portfolio_churn_clos2y_over_active,w720_risky_type_share,w720_prolongation_stress,w720_num_new,w720_vintage_fraction
2288,102681,3.0,0.0,3.0,1.000000,-0.000000,1.0,90643.50,103495.50,0.875821,...,77791.5,0.0,consumer credit,0.951984,0.000000,0.000000,0.000000,0.0,4.0,0.800000
13322,115531,3.0,0.0,3.0,1.000000,0.636514,1.0,79070.40,113724.00,0.695283,...,22455.9,0.0,credit card,0.619860,0.200000,0.250000,0.000000,0.0,5.0,0.555556
15397,117920,3.0,0.0,3.0,1.000000,0.636514,1.0,35118.00,49500.00,0.709455,...,9720.0,0.0,microloan,0.542758,0.266667,1.500000,0.466667,0.0,15.0,0.625000
15758,118326,10.0,1.0,11.0,0.909091,0.304636,1.0,318496.50,589500.00,0.540282,...,22500.0,0.0,microloan,0.369522,0.083333,1.000000,0.916667,0.0,36.0,0.765957
18922,121997,3.0,1.0,4.0,0.750000,-0.000000,1.0,4500.00,33750.00,0.133333,...,0.0,0.0,microloan,1.000000,0.000000,0.250000,1.000000,0.0,5.0,0.625000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262695,406014,4.0,0.0,4.0,1.000000,0.693147,1.0,420659.82,437309.82,0.961926,...,135000.0,0.0,credit card,0.472614,0.000000,0.500000,0.000000,0.0,6.0,0.545455
272084,416861,3.0,0.0,3.0,1.000000,1.098612,1.0,35995.50,35995.95,0.999987,...,0.0,0.0,credit card,0.284250,0.130435,0.916667,0.391304,0.0,23.0,0.696970
276720,422284,3.0,0.0,3.0,1.000000,-0.000000,1.0,46944.00,94500.00,0.496762,...,25200.0,0.0,consumer credit,0.702490,0.000000,0.000000,0.000000,0.0,8.0,0.571429
280798,427060,3.0,0.0,3.0,1.000000,-0.000000,1.0,72000.00,148500.00,0.484848,...,72000.0,0.0,microloan,0.652598,0.000000,0.000000,0.750000,0.0,8.0,0.421053


In [88]:
# write bureau feature to parquet:
bureau_features.to_parquet(rf'{data_dir}\data\processed\bureau_features.parquet')

In [89]:
bureau_features = pd.read_parquet(rf'{data_dir}\data\processed\bureau_features.parquet')

In [90]:
bureau_features.shape

(305811, 196)

In [91]:
bureau_features['SK_ID_CURR'].drop_duplicates().count()

305811

In [92]:
df_bureau['SK_ID_CURR'].drop_duplicates().count()

305811

In [93]:
# Check bureau features for null 100%

bureau_features.isnull().mean().sort_values(ascending=False).head(20)

w30_avg_closed_duration                   0.996007
w30_high_util_card_any                    0.993669
w90_avg_closed_duration                   0.983281
w30_debt_concentration_active             0.970943
w30_annuity_over_total_debt               0.970926
w30_duration_var                          0.968409
w30_avg_days_remaining                    0.968409
w30_portfolio_churn_clos2y_over_active    0.966783
w30_avg_overdue_active                    0.966783
w30_expo_growth_rate                      0.964465
w30_prolongation_stress                   0.964419
w30_utilization_portfolio                 0.964419
w30_expo_concentration                    0.964419
w30_chronic_delinquent                    0.964318
w30_last_loan_overdue_amt                 0.964318
w30_short_term_risk                       0.964318
w30_recency_weighted_debt_tau365          0.964318
w30_last_loan_debt                        0.964318
w30_laddering_share                       0.964318
w30_risky_type_share           

In [87]:
bureau_features['w360_num_new'].unique()

array([ 2.,  1., nan,  4.,  3.,  5.,  6.,  9.,  8.,  7., 11., 10., 17.,
       14., 13., 12., 15., 16., 34., 18., 23., 79., 28., 19., 22., 20.,
       32., 24., 30., 25., 21., 26., 36., 35., 29., 33., 27., 41.])

## BUREAU_BALANCE

In [30]:
bbp_test = bb_features_per_account(df_bureau_bal.loc[df_bureau_bal['SK_BUREAU_ID'].isin([5625646,5625647])],months_windows=[3])

In [34]:
agg_bb_to_customer(df_bureau[df_bureau['SK_ID_CURR']==100221],bbp_test)['bb_w3_dpd_max__max']

0    5.0
Name: bb_w3_dpd_max__max, dtype: float64

In [42]:
compute_bureau_balance_features(
    df_bureau.loc[df_bureau['SK_ID_CURR']==100221, ['SK_BUREAU_ID','SK_ID_CURR']],
    df_bureau_bal.loc[df_bureau_bal['SK_BUREAU_ID'].isin(df_bureau[df_bureau['SK_ID_CURR']==100221]['SK_BUREAU_ID'].unique())],
    months_windows=[3]
    )[['SK_ID_CURR','bb_w3_dpd_max__max']]

,SK_ID_CURR,bb_w3_dpd_max__max
0,100221,5.0


In [43]:
bureau_bal_features = compute_bureau_balance_features(df_bureau[['SK_BUREAU_ID','SK_ID_CURR']], df_bureau_bal, months_windows=[3,6,12,24])

In [18]:
bureau_bal_features

,SK_ID_CURR,bb_bb_last_month__mean,bb_bb_last_month__max,bb_bb_last_month__sum,bb_bb_last_status_code__mean,bb_bb_last_status_code__max,bb_bb_last_status_code__sum,bb_bb_last_status_is_closed__mean,bb_bb_last_status_is_closed__max,bb_bb_last_status_is_closed__sum,...,bb_w24_status_2_share__sum,bb_w24_status_3_share__mean,bb_w24_status_3_share__max,bb_w24_status_3_share__sum,bb_w24_status_4_share__mean,bb_w24_status_4_share__max,bb_w24_status_4_share__sum,bb_w24_status_5_share__mean,bb_w24_status_5_share__max,bb_w24_status_5_share__sum
0,100001,0.000000,0.0,0.0,-0.428571,1.0,-3.0,0.571429,1.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100002,-15.500000,0.0,-124.0,-0.750000,0.0,-6.0,0.750000,1.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100003,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0
3,100004,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0
4,100005,0.000000,0.0,0.0,-0.500000,0.0,-1.0,0.333333,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305806,456249,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0
305807,456250,0.000000,0.0,0.0,-0.500000,0.0,-1.0,0.333333,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
305808,456253,0.000000,0.0,0.0,-1.000000,-1.0,-3.0,0.750000,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
305809,456254,0.000000,0.0,0.0,-1.000000,-1.0,-1.0,1.000000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
bureau_bal_features.to_parquet(rf'{data_dir}\data\processed\bureau_bal_features.parquet')

In [50]:
bureau_bal_features = pd.read_parquet(rf'{data_dir}\data\processed\bureau_bal_features.parquet')

In [94]:
bureau_bal_features.loc[bureau_bal_features['SK_ID_CURR']==100221,['bb_w3_status_max__max']]

,bb_w3_status_max__max
194,5.0


In [52]:
bureau_bal_features.shape

(305811, 238)

In [53]:
bureau_bal_features['SK_ID_CURR'].drop_duplicates().count()

305811

## APPLICATION

### Preprocessing

In [55]:
list_flag_cols_notnumeric = [ft for ft in df_train.columns.tolist() if str(ft).startswith("FLAG_") & (not str(df_train[ft][0]).isnumeric())]

In [56]:
list_flag_cols_notnumeric

['FLAG_OWN_CAR', 'FLAG_OWN_REALTY']

In [57]:
df_train[list_flag_cols_notnumeric]

,FLAG_OWN_CAR,FLAG_OWN_REALTY
0,N,Y
1,N,N
2,Y,Y
3,N,Y
4,N,Y
...,...,...
307506,N,N
307507,N,Y
307508,N,Y
307509,N,Y


In [58]:
for ft in list_flag_cols_notnumeric:
    df_train[ft] = np.where(df_train[ft]=="Y",1,0)

In [59]:
df_train[list_flag_cols_notnumeric]

,FLAG_OWN_CAR,FLAG_OWN_REALTY
0,0,1
1,0,0
2,1,1
3,0,1
4,0,1
...,...,...
307506,0,0
307507,0,1
307508,0,1
307509,0,1


### Calculate features

In [63]:
application_feature = compute_application_features(df_train)

In [ ]:
# application_feature.to_parquet(rf'{data_dir}\processed\application_features.parquet')

In [19]:
application_feature = pd.read_parquet(rf'{data_dir}\processed\application_features.parquet')

In [64]:
application_feature.head()

,SK_ID_CURR,AGE_YEARS,EMPLOY_YEARS,REG_YEARS_AGO,ID_PUBLISH_YEARS_AGO,PHONE_CHANGE_YEARS_AGO,CREDIT_TO_INCOME,ANNUITY_TO_INCOME,ANNUITY_TO_CREDIT,GOODS_TO_CREDIT,...,REQ_RECENT_RATIO,REQ_YEAR_ONLY,ASSET_FLAGS_SUM,DOCS_COUNT,DOCS_MISSING_RATIO,HOUR_SIN,HOUR_COS,HOUR_LATE_FLAG,ENV_MISSING_CNT,ENV_COVERAGE
0,100002,25.920548,1.745205,9.994521,5.808219,3.106849,2.007889,0.121978,0.060749,0.863262,...,0.0,1.0,1.0,1,0.95,0.500000,-0.866025,0.0,0,1.0
1,100003,45.931507,3.254795,3.249315,0.797260,2.268493,4.790750,0.132217,0.027598,0.873211,...,NaN,0.0,0.0,1,0.95,0.258819,-0.965926,0.0,0,1.0
2,100004,52.180822,0.616438,11.671233,6.934247,2.232877,2.000000,0.100000,0.050000,1.000000,...,NaN,0.0,2.0,0,1.00,0.707107,-0.707107,0.0,47,0.0
3,100006,52.068493,8.326027,26.939726,6.676712,1.690411,2.316167,0.219900,0.094941,0.949845,...,NaN,0.0,1.0,1,0.95,-0.965926,-0.258819,0.0,47,0.0
4,100007,54.608219,8.323288,11.810959,9.473973,3.030137,4.222222,0.179963,0.042623,1.000000,...,NaN,0.0,1.0,1,0.95,0.258819,-0.965926,0.0,47,0.0


In [95]:
application_feature.shape

(307511, 35)

In [96]:
application_feature['SK_ID_CURR'].drop_duplicates().count()

307511

## COMBINATION

In [97]:
final_df = application_feature.merge(bureau_features, on='SK_ID_CURR', how='left').merge(bureau_bal_features, on='SK_ID_CURR', how='left')

In [98]:
# write final_df to parquet
final_df.to_parquet(rf'{data_dir}\data\processed\final_df.parquet')

In [99]:
# check missing rate of final_df and output to a dataframe with two columns: feature name and missing rate
missing_rate = pd.DataFrame(data=final_df.isnull().sum() / final_df.shape[0],columns=['missing_rate']).reset_index(names='feature_name')

In [100]:
# filter missing_rate to show only features with missing rate > 0.7
missing_rate[missing_rate['missing_rate']>0.7]

,feature_name,missing_rate
35,w30_n_active_loans,0.969494
36,w30_n_closed_loans,0.969494
37,w30_n_total_loans,0.969494
38,w30_active_ratio,0.969494
39,w30_type_diversity_shannon,0.969494
...,...,...
459,bb_w24_status_3_share__max,0.704911
461,bb_w24_status_4_share__mean,0.704911
462,bb_w24_status_4_share__max,0.704911
464,bb_w24_status_5_share__mean,0.704911
